In [1]:
pip install transformers

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!git clone https://github.com/Anthony-Alridge/QualityEstimation
%cd 'QualityEstimation'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: destination path 'QualityEstimation' already exists and is not an empty directory.
/content/QualityEstimation


In [3]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from torch.utils import data

In [0]:
f = open('data/en-zh/train.enzh.src') # Open file on read mode
lines_en = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/train.enzh.mt') # Open file on read mode
lines_de = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/train.enzh.scores') # Open file on read mode
scores = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

In [0]:
tokeniser = ppb.XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

datas = []
for i,j in zip(lines_en, lines_de):
  if i is '':break
  datas.append(tokeniser.encode(i,text_pair=j,max_length=30,pad_to_max_length=True))
data_tensor = torch.LongTensor(datas)



In [6]:
model = ppb.XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels = 1)
# model.config.num_labels = 1
model.cuda()

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [0]:
scores_ls = []
for s in scores:
  if s is None or s is '':break
  scores_ls.append(float(s))

labels = torch.FloatTensor(scores_ls)

In [0]:
# print(labels.shape)
# print(data_tensor.shape)


In [0]:
my_dataset = data.TensorDataset(data_tensor,labels) # create your datset
my_dataloader = data.DataLoader(my_dataset,batch_size=16,shuffle=True) 

In [9]:
epochs=10
device=torch.device('cuda')
dtype = torch.long
print_every = 100
optimizer = torch.optim.Adamax(model.parameters())
for e in range(epochs):
        for t, (x, y) in enumerate(my_dataloader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.float)
            outputs = model(x, labels=y)
            loss = outputs[0]
            optimizer.zero_grad()

            loss.backward()

            # Update the parameters of the model using the gradients
            optimizer.step()

            if t % print_every == 0:
                print('Epoch: %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                #check_accuracy(loader_val, model)
                print()

Epoch: 0, Iteration 0, loss = 1.1017



RuntimeError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = '/content/drive/My Drive/NLP Chinese Models/xlm_roberta/'
model.save_pretrained(path)  # save
tokeniser.save_pretrained(path)  # save